##Using CLIP to generate CLIP vectors for Image-MemeText Pairs

In [1]:
!pip install --upgrade jupyter_http_over_ws>=0.0.7 && jupyter serverextension enable --py jupyter_http_over_ws

Enabling: jupyter_http_over_ws
- Writing config: /root/.jupyter
    - Validating...
      jupyter_http_over_ws  OK


In [2]:
#!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-oxizj_ca
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-oxizj_ca
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369409 sha256=68805600889206d5227e9369fee5dfe393569760b52bee5cf820177664a76aa2
  Stored in directory: /tmp/pip-ephem-wheel-cache-evie4rvy/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip


In [3]:
import numpy as np
import torch
from pkg_resources import packaging

print("Torch version:", torch.__version__)


Torch version: 1.12.1+cu113


In [4]:
import clip

clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [5]:
clip_model, preprocess = clip.load("ViT-B/32")
clip_model.cuda().eval()

100%|███████████████████████████████████████| 338M/338M [00:04<00:00, 72.0MiB/s]


CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [6]:
import os
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np, pandas as pd
np.random.seed(42)
from collections import OrderedDict
import torch

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [7]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Data/

Mounted at /content/drive
/content/drive/My Drive/Data


In [8]:
IMAGES_DIR = '/content/drive/My Drive/Data/memotion_dataset_7k/en_images/'

labels_df_A = pd.read_csv('/content/drive/My Drive/Data/memotion_dataset_7k/Labels_A.csv')
labels_df_A.head()

descriptions = {}
for row in labels_df_A.index:
  descriptions[labels_df_A.image_name.tolist()[row]] = labels_df_A.text_corrected.tolist()[row]

print(descriptions)

{'image_1.jpg': 'LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIKUT TREND PLAY THE 10 YEARS CHALLENGE AT FACEBOOK imgflip.com', 'image_2.jpeg': 'The best of #10 YearChallenge! Completed in less the 4 years. Kudus to @narendramodi ji 8:05 PM - 16 Jan 2019 from Mumbai  India', 'image_3.JPG': "Sam Thorne @Strippin ( Follow Follow Saw everyone posting these 2009 vs 2019 pics so here's mine 6:23 PM - 12 Jan 2019 O 636 Retweets 3 224 LIKES 65 636 3.2K", 'image_5.png': '10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious 10 Year Challenge Memes | What is #10 Year Challenge?', 'image_6.jpg': '1998: "Don\'t get in car with strangers" 2008: "Don\'t meet people from the internet alone." MemeCenter.com 2019: UBER.. Order yourself a stranger from the internet to get into a car with alone.', 'image_9.jpg': 'Fornite died in 10 months but Minecraft never died in 10 years I am so proud of this community', 'image_10.png': "FACEBOOK '10 YEAR CHALLENGE': A PLOY OR A SIMPLE MEME? mo by eXo Hạ Minh Le 18 Jan 2018",

In [10]:
original_images = []
images = []
texts = []

for filename in [filename for filename in os.listdir('/content/drive/My Drive/Data/memotion_dataset_7k/en_images/')][3000:len(os.listdir('/content/drive/My Drive/Data/memotion_dataset_7k/en_images/'))+1]:
  name = os.path.splitext(filename)[0]
  print(os.path.join('/content/drive/My Drive/Data/memotion_dataset_7k/en_images/', filename))
  image = Image.open(os.path.join('/content/drive/My Drive/Data/memotion_dataset_7k/en_images/', filename)).convert("RGB")
  print(name, descriptions[filename])
  original_images.append(image)
  images.append(preprocess(image))
  texts.append(descriptions[filename])

Streaming output truncated to the last 5000 lines.
/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_2005.jpeg
image_2005 HOLY CRAP WE CAN'T STOP DONALD TRUMP STAND. ASIDE  LITTLE MEN  I GOT THIS
/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_2006.jpg
image_2006 I'M HILLARY CLINTON. MY LIFE IS A DANK MEME AND I GOT DANK MEMES COMIN OUT MY BUTTHOLE makeameme.org
/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_2008.png
image_2008 i want to protect toddlers from guns its But i have no problem murdering a 9 month old fetus Make a Meтet Hillary's logic.
/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_2009.png
image_2009 BELIEVES SNOWDEN SHOULD BE IMPRISONED BECAUSE HE COMPROMISED AMERICAN SAFETY AND SECURITY. PUTS TOP SECRET INFORMATION ON A PERSONAL E-MAIL SERVER IN HER HOME. made on imgur
/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_2010.jpg
image_2010 It's hip hop... You wouldn't get it... Just take th

In [11]:
len(images)

2544

In [13]:
indices = [x for x in range(0, len(images), 2)]
print(indices)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102, 104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128, 130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152, 154, 156, 158, 160, 162, 164, 166, 168, 170, 172, 174, 176, 178, 180, 182, 184, 186, 188, 190, 192, 194, 196, 198, 200, 202, 204, 206, 208, 210, 212, 214, 216, 218, 220, 222, 224, 226, 228, 230, 232, 234, 236, 238, 240, 242, 244, 246, 248, 250, 252, 254, 256, 258, 260, 262, 264, 266, 268, 270, 272, 274, 276, 278, 280, 282, 284, 286, 288, 290, 292, 294, 296, 298, 300, 302, 304, 306, 308, 310, 312, 314, 316, 318, 320, 322, 324, 326, 328, 330, 332, 334, 336, 338, 340, 342, 344, 346, 348, 350, 352, 354, 356, 358, 360, 362, 364, 366, 368, 370, 372, 374, 376, 378, 380, 382, 384, 386, 388, 390, 392, 394, 396, 398, 400, 402, 404, 406, 408, 410, 412, 414, 416, 418, 420,

In [14]:
mt = torch.empty((1,1024), dtype=torch.int64).cuda()
mt

tensor([[        6543851520,               8064,                  0,
          ...,  -2826883050375946,  -1413989133518341,
         -51713012561608961]], device='cuda:0')

In [15]:
for i in range(0,len(images),2):
  image_input = torch.tensor(np.stack(images[i:i+2])).cuda()
  text_tokens = clip.tokenize([desc for desc in texts[i:i+2]], truncate=True).cuda()
  #print(image_input.shape)
  #print(text_tokens.shape)
  with torch.no_grad():
    image_features = clip_model.encode_image(image_input).float()
    #print(image_features.shape)
    text_features = clip_model.encode_text(text_tokens).float()
    #print(text_features.shape)
  
  concat_features = torch.cat((image_features, text_features), 1)
  mt = torch.cat((mt, concat_features), 0)
  print(mt.shape)

torch.Size([3, 1024])
torch.Size([5, 1024])
torch.Size([7, 1024])
torch.Size([9, 1024])
torch.Size([11, 1024])
torch.Size([13, 1024])
torch.Size([15, 1024])
torch.Size([17, 1024])
torch.Size([19, 1024])
torch.Size([21, 1024])
torch.Size([23, 1024])
torch.Size([25, 1024])
torch.Size([27, 1024])
torch.Size([29, 1024])
torch.Size([31, 1024])
torch.Size([33, 1024])
torch.Size([35, 1024])
torch.Size([37, 1024])
torch.Size([39, 1024])
torch.Size([41, 1024])
torch.Size([43, 1024])
torch.Size([45, 1024])
torch.Size([47, 1024])
torch.Size([49, 1024])
torch.Size([51, 1024])
torch.Size([53, 1024])
torch.Size([55, 1024])
torch.Size([57, 1024])
torch.Size([59, 1024])
torch.Size([61, 1024])
torch.Size([63, 1024])
torch.Size([65, 1024])
torch.Size([67, 1024])
torch.Size([69, 1024])
torch.Size([71, 1024])
torch.Size([73, 1024])
torch.Size([75, 1024])
torch.Size([77, 1024])
torch.Size([79, 1024])
torch.Size([81, 1024])
torch.Size([83, 1024])
torch.Size([85, 1024])
torch.Size([87, 1024])
torch.Size([89,

In [16]:
row_exclude = 1
mt = torch.cat((mt[:row_exclude],mt[row_exclude+1:]))
mt.shape

torch.Size([2544, 1024])

In [17]:
torch.save(mt, 'tensor_1_3000.pt')